In [3]:
import os
os.environ["OPENCV_VIDEOIO_MSMF_ENABLE_HW_TRANSFORMS"] = "0"
import cv2
import numpy as np
import glob
import time
from IPython.display import clear_output
import json

In [2]:
def getAvailibleCameras():
    available_cameras = []
    for i in range(10):
        try:
            camera = cv2.VideoCapture(i)
            if camera.isOpened():
                print("Camera index:", i)
                camera.release()
                available_cameras.append(i)
        except:
            pass
    return available_cameras

### https://markhedleyjones.com/projects/calibration-checkerboard-collection
Used a 8 x 6 Checker board with 25mm titles an A4 paper in full pages without margin

In [4]:
# cap = cv2.VideoCapture(0, cv2.CAP_MSMF) # u can use cv2.CAP_DSHOW / cv2.CAP_MSMF / cv2.CAP_VFW / cv2.CAP_ANY

cap = cv2.VideoCapture(3,cv2.CAP_MSMF)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 3840)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 2160)
#1080p
# q
if not cap.isOpened():
    raise Exception("Could not open video device")

while True:
    #capture an image and show it 
    ret, frame = cap.read()
    if not ret:
        raise Exception("Could not read frame")
    #show the image
    resized = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
    cv2.imshow("resized", resized)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cv2.destroyAllWindows()


In [5]:
cap = cv2.VideoCapture(3,cv2.CAP_MSMF)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 3840)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 2160)


save_path = 'calibration_images_close_up'
if not os.path.exists(save_path):
    os.mkdir(save_path)
img_counter = 20
#checkerboars settings
CHECKERBOARD = (7,7)
do_wait = False
scale = 0.25
while True:
    ret, frame = cap.read()
    if ret:
        gray = cv2.cvtColor(frame ,cv2.COLOR_BGR2GRAY)

        gray = cv2.resize(gray, (0, 0), fx=scale, fy=scale)

        result, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
        if result == True:
            cv2.imwrite(os.path.join(save_path, 'calibration_image_{}.png'.format(img_counter)), frame)
            img_counter += 1
            frame = cv2.resize(frame, (0, 0), fx=scale, fy=scale)
            cv2.drawChessboardCorners(frame, CHECKERBOARD, corners, result)
            cv2.imshow("frame", frame)
        else:
            frame = cv2.resize(frame, (0, 0), fx=scale, fy=scale)
            cv2.imshow("frame", frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break
cap.release()
cv2.destroyAllWindows()
        

In [6]:
img_path = save_path
imgs_path = glob.glob(img_path + '/*.png')
for img_path in imgs_path:
    print(img_path)

calibration_images_close_up\calibration_image_100.png
calibration_images_close_up\calibration_image_101.png
calibration_images_close_up\calibration_image_102.png
calibration_images_close_up\calibration_image_103.png
calibration_images_close_up\calibration_image_104.png
calibration_images_close_up\calibration_image_105.png
calibration_images_close_up\calibration_image_106.png
calibration_images_close_up\calibration_image_107.png
calibration_images_close_up\calibration_image_108.png
calibration_images_close_up\calibration_image_109.png
calibration_images_close_up\calibration_image_110.png
calibration_images_close_up\calibration_image_111.png
calibration_images_close_up\calibration_image_112.png
calibration_images_close_up\calibration_image_113.png
calibration_images_close_up\calibration_image_114.png
calibration_images_close_up\calibration_image_115.png
calibration_images_close_up\calibration_image_116.png
calibration_images_close_up\calibration_image_117.png
calibration_images_close_up\

In [7]:
# Defining the dimensions of checkerboard
CHECKERBOARD = (7,7)
CHECKERBOARD_CELL_SIZE = 0.004 # in meters
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)
 
# Creating vector to store vectors of 3D points for each checkerboard image
objpoints = []
# Creating vector to store vectors of 2D points for each checkerboard image
imgpoints = [] 
 
 
# Defining the world coordinates for 3D points
objp = np.zeros((1, CHECKERBOARD[0] * CHECKERBOARD[1], 3), np.float32)
objp[0,:,:2] = np.mgrid[0:CHECKERBOARD[0], 0:CHECKERBOARD[1]].T.reshape(-1, 2)
prev_img_shape = None

for fname in imgs_path:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    # Find the chess board corners
    # If desired number of corners are found in the image then ret = true
    ret, corners = cv2.findChessboardCorners(gray, CHECKERBOARD, cv2.CALIB_CB_ADAPTIVE_THRESH + cv2.CALIB_CB_FAST_CHECK + cv2.CALIB_CB_NORMALIZE_IMAGE)
     
    """
    If desired number of corner are detected,
    we refine the pixel coordinates and display 
    them on the images of checker boardq
    """
    if ret == True:
        objpoints.append(objp)
        # refining pixel coordinates for given 2d points.
        corners2 = cv2.cornerSubPix(gray, corners, (11,11),(-1,-1), criteria)
         
        imgpoints.append(corners2)
 
        # Draw and display the corners
        img = cv2.drawChessboardCorners(img, CHECKERBOARD, corners2, ret)
        # print 'press any key to continue'
        img = cv2.putText(img, 'press any key to continue', (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    cv2.imshow('img',img)
    cv2.waitKey(0)
 
cv2.destroyAllWindows()
 
h,w = img.shape[:2]
 
"""
Performing camera calibration by 
passing the value of known 3D points (objpoints)
and corresponding pixel coordinates of the 
detected corners (imgpoints)
"""
ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)
 
print("Camera matrix : \n")
print(mtx)
print("dist : \n")
print(dist)
print("rvecs : \n")
print(rvecs)
print("tvecs : \n")
print(tvecs)

Camera matrix : 

[[1.74165212e+03 0.00000000e+00 1.87710244e+03]
 [0.00000000e+00 1.74782347e+03 1.12091960e+03]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
dist : 

[[-0.01594486  0.02017146  0.00325647 -0.00355718 -0.0114934 ]]
rvecs : 

(array([[-0.08399076],
       [ 0.01395688],
       [ 1.05322892]]), array([[-0.0748348 ],
       [ 0.0118204 ],
       [ 1.03900201]]), array([[-0.07506362],
       [ 0.01241343],
       [ 1.03874066]]), array([[-0.03664348],
       [-0.03173698],
       [ 1.2512417 ]]), array([[-0.03343112],
       [-0.02915682],
       [ 1.25863459]]), array([[-0.03480192],
       [-0.02788094],
       [ 1.25773083]]), array([[-0.03047409],
       [-0.02533182],
       [ 1.25633995]]), array([[-0.03467142],
       [-0.02169013],
       [ 1.25662529]]), array([[-0.03118425],
       [-0.02186336],
       [ 1.25696771]]), array([[-0.0330052 ],
       [-0.01726762],
       [ 1.2562694 ]]), array([[-0.02920928],
       [-0.01560476],
       [ 1.24986782]]), array

In [8]:
#save the camera matrix and distortion coefficients
with open(f'{save_path}/calibration_data.json', 'w') as outfile:
    json.dump({'mtx': mtx.tolist(), 'dist': dist.tolist()}, outfile)
#reconstruct the camera matrix and distortion coefficients
with open(f'{save_path}/calibration_data.json', 'r') as infile:
    reconstruction = json.load(infile)
    new_mtx = np.array(reconstruction['mtx'])
    new_dist = np.array(reconstruction['dist'])
cv2.destroyAllWindows()
print('done')

done
